In [ ]:
import spikeextractors as se
import hither as hi
import numpy as np
import kachery_p2p as kp
import labbox_ephys as le
from labbox_ephys import sorting_views
from labbox_ephys import recording_views

In [ ]:
def prepare_recording_sorting():
    # Simulate a recording (toy example)
    recording, sorting = se.example_datasets.toy_example(duration=50, num_channels=4, K=4, seed=1)
    # Write data to temporary files and store in a labbox-ephys-compatible format
    with hi.TemporaryDirectory() as tmpdir:
        raw_fname = tmpdir + '/raw.dat'
        traces: np.ndarray = recording.get_traces()
        traces = (traces / np.max(np.abs(traces)) * 10000).astype(np.int16)
        se.BinDatRecordingExtractor.write_recording(recording, raw_fname, time_axis=0, dtype=np.int16)
        bin_uri = kp.store_file(raw_fname)
        channel_ids = [int(id) for id in recording.get_channel_ids()]
        channel_map = dict()
        channel_positions = dict()
        locations = recording.get_channel_locations()
        for id, ii in enumerate(channel_ids):
            channel_map[str(id)] = ii
            channel_positions[str(id)] = locations[ii].tolist()
        recording_obj = {
            'recording_format': 'bin1',
            'data': {
                'raw': bin_uri,
                'raw_num_channels': recording.get_num_channels(),
                'num_frames': recording.get_num_frames(),
                'samplerate': recording.get_sampling_frequency(),
                'channel_ids': recording.get_channel_ids(),
                'channel_map': channel_map,
                'channel_positions': channel_positions
            }
        }
        R = le.LabboxEphysRecordingExtractor(recording_obj)

        sorting_path = tmpdir + '/sorting.h5'
        le.extractors.H5SortingExtractorV1.write_sorting(sorting, sorting_path)
        S = le.LabboxEphysSortingExtractor({
            'sorting_format': 'h5_v1',
            'data': {
                'h5_path': kp.store_file(sorting_path)
            }
        })
    return R, S

In [ ]:
recording, sorting = prepare_recording_sorting()

In [ ]:
sorting_views.AverageWaveformsNew(sorting=sorting, recording=recording)

In [ ]:
sorting_views.ElectrodeGeometry(sorting=sorting, recording=recording)

In [ ]:
sorting_views.Autocorrelograms(sorting=sorting, recording=recording)

In [ ]:
sorting_views.Timeseries(sorting=sorting, recording=recording)

In [ ]:
recording_views.Timeseries(recording=recording)